In [ ]:
pip install phe

  Created wheel for phe: filename=phe-1.4.0-py2.py3-none-any.whl size=37364 sha256=0a8085ecd29d930d36f1dfc3eeb636d0de1b855c1cd290c62355f6e2d3537950
  Stored in directory: /root/.cache/pip/wheels/f8/dc/36/dcb6bf0f1b9907e7b710ace63e64d08e7022340909315fdea4
Successfully built phe


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.layers import Input, Dense, GaussianNoise
from keras.models import Model
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
import random as rn
import sys
import matplotlib.pyplot as plt
from phe import paillier

In [ ]:
M = 16
k = int(np.log2(M))
print('M:{} k:{}'.format(M,k))

M:16 k:4


In [ ]:
N = 10000
label = np.random.randint(M,size=N)
N_val = 1500
val_label = np.random.randint(M,size=N_val)
print(type(label))
label[:10]

<class 'numpy.ndarray'>


array([ 3,  4,  1, 12,  0,  0,  7,  1, 13, 15])

In [ ]:
data = []
for i in label:
    temp = np.zeros(M)
    temp[i] = 1
    data.append(temp)

val_data = []
for i in val_label:
    temp = np.zeros(M)
    temp[i] = 1
    val_data.append(temp)

In [ ]:
data = np.array(data)
val_data = np.array(val_data)
print(data.shape)

(10000, 16)


In [ ]:
R = 4/7
n_channel = 7
input_signal = Input(shape=(M,))
encoded = Dense(M, activation='relu')(input_signal)
encoded1 = Dense(n_channel, activation='linear')(encoded)
encoded2 = BatchNormalization()(encoded1)

EbNo_train = 5.01187 #  coverted 7 db of EbNo
encoded3 = GaussianNoise(np.sqrt(1/(2*R*EbNo_train)))(encoded2)

decoded = Dense(M, activation='relu')(encoded3)
decoded1 = Dense(M, activation='softmax')(decoded)

autoencoder = Model(input_signal, decoded1)
#sgd = SGD(lr=0.001)
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
#keras.utils.plot_model(autoencoder, show_shapes=True)

In [ ]:
print (autoencoder.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16)]              0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 119       
_________________________________________________________________
batch_normalization (BatchNo (None, 7)                 28        
_________________________________________________________________
gaussian_noise (GaussianNois (None, 7)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                128       
_________________________________________________________________
dense_3 (Dense)              (None, 16)               

In [ ]:
autoencoder.fit(data, data,
                epochs=20,
                batch_size=100,
                validation_data=(val_data, val_data))

Epoch 1/20
100/100 [==============================] - 0s 3ms/step - loss: 2.2895 - val_loss: 2.4462
Epoch 2/20
100/100 [==============================] - 0s 1ms/step - loss: 1.2582 - val_loss: 1.7084
Epoch 3/20
100/100 [==============================] - 0s 1ms/step - loss: 0.5747 - val_loss: 0.7479
Epoch 4/20
100/100 [==============================] - 0s 1ms/step - loss: 0.2201 - val_loss: 0.1981
Epoch 5/20
100/100 [==============================] - 0s 1ms/step - loss: 0.1008 - val_loss: 0.0594
Epoch 6/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0567 - val_loss: 0.0249
Epoch 7/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0374 - val_loss: 0.0140
Epoch 8/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0257 - val_loss: 0.0090
Epoch 9/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0195 - val_loss: 0.0063
Epoch 10/20
100/100 [==============================] - 0s 1ms/step - loss: 0.0145 - val_loss: 0.0047

Loading the trained model and validating the model using validation data.

In [ ]:
encoder = Model(input_signal, encoded2)

In [ ]:
encoded_input = Input(shape=(n_channel,))

deco = autoencoder.layers[-2](encoded_input)
deco = autoencoder.layers[-1](deco)
# create the decoder model
decoder = Model(encoded_input, deco)

In [ ]:
N = 45000
test_label = np.random.randint(M,size=N)
print('Input given is:',test_label)
test_data = []

for i in test_label:
    temp = np.zeros(M)
    temp[i] = 1
    test_data.append(temp)


Input given is: [ 6  6 13 ... 10  5  7]


In [ ]:
test_data = np.array(test_data)

In [ ]:
public_key, private_key = paillier.generate_paillier_keypair()    # Generating Encryption Keys

In [ ]:
encoded_signal = encoder.predict(test_data)
decrypted = pd.DataFrame(0, index=range(10), columns=range(encoded_signal.shape[1]))
encrypted = pd.DataFrame(0, index=range(10), columns=range(encoded_signal.shape[1]))

def encrypt_data(sample):
    return public_key.encrypt(float(sample))

def decrypt_data(sample):
    return private_key.decrypt(sample)


def frange(x, y, jump):
  while x < y:
    yield x
    x += jump

In [ ]:
ber = [None]
EbNodB_range = 1
EbNo=10.0**(1/10.0)
noise_std = np.sqrt(1/(2*R*EbNo))
noise_mean = 0
no_errors = []
nn = N
    
noise = noise_std * np.random.randn(nn,n_channel)
encoded_signal = encoder.predict(test_data)  
print("\nOriginal Data:",np.squeeze(test_data[:1]).astype(int))      
pencoded = pd.DataFrame(encoded_signal)
pencoded = pencoded[:1]
pnoise = pd.DataFrame(noise)
print("\nEncoded Data:",pencoded[0])
encrypted = pencoded.applymap(encrypt_data)
print("\nEncoded & Encrypted Data:")
print(encrypted[0])

nencrypted = encrypted + pnoise[:1]

decrypted = nencrypted.applymap(decrypt_data)
final_signal = decrypted.to_numpy()
print("\nDecrypted Data:")
print(final_signal[0])
pred_final_signal =  np.squeeze(decoder.predict(final_signal))
print("\nDecoded Data:",'\n', pred_final_signal)

pred_final_signal = [int(np.maximum(1, pred_final_signal[x])) if pred_final_signal[x] >.1 else int(np.minimum(0, pred_final_signal[x])) for x in range(0,len(pred_final_signal)) ]
print('\nThe received data:',np.array(pred_final_signal))


Original Data: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]

Encoded Data: 0    1.137766
Name: 0, dtype: float32

Encoded & Encrypted Data:
0    <phe.paillier.EncryptedNumber object at 0x7f2f...
Name: 0, dtype: object

Decrypted Data:
[ 1.04432641  1.94505073  0.11033932 -3.01818324  0.49540513 -0.11222995
 -1.02552479]

Decoded Data: 
 [1.4252451e-07 1.0033128e-06 6.8766174e-05 5.4446189e-04 2.3073972e-05
 8.0650864e-04 9.9542725e-01 1.1171099e-07 6.7534768e-07 8.1151702e-05
 8.4924599e-05 3.6593468e-05 2.0000974e-03 5.7292846e-04 7.5956586e-07
 3.5163143e-04]

The received data: [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]


In [ ]:
encoded_signal = encoder.predict(test_data)
decrypted = pd.DataFrame(0, index=range(10), columns=range(encoded_signal.shape[1]))
encrypted = pd.DataFrame(0, index=range(10), columns=range(encoded_signal.shape[1]))

def encrypt_data(sample):
    return public_key.encrypt(float(sample))

def decrypt_data(sample):
    return private_key.decrypt(sample)


def frange(x, y, jump):
  while x < y:
    yield x
    x += jump

EbNodB_range = list(frange(-4,4,1))

ber = [None]*len(EbNodB_range)
for n in range(0,len(EbNodB_range)):
    EbNo=10.0**(EbNodB_range[n]/10.0)
    noise_std = np.sqrt(1/(2*R*EbNo))
    noise_mean = 0
    no_errors = []
    nn = N
    
    noise = noise_std * np.random.randn(nn,n_channel)
    encoded_signal = encoder.predict(test_data)  
    print("\nOriginal Data:")           
    print(test_data[:10])
    pencoded = pd.DataFrame(encoded_signal)
    pencoded = pencoded[:10]
    pnoise = pd.DataFrame(noise)
    print("\nEncoded Data:")
    print(pencoded[0])
    encrypted = pencoded.applymap(encrypt_data)
    print("\nEnocded & Encrypted Data:")
    print(encrypted[0])

    nencrypted = encrypted + pnoise[:10]

    decrypted = nencrypted.applymap(decrypt_data)
    final_signal = decrypted.to_numpy()
    print("\nDecrypted Data:")
    print(final_signal[0])
    pred_final_signal =  decoder.predict(final_signal)
    print("\nDecoded Data:")
    print(pred_final_signal[:10])

    pred_output = np.argmax(pred_final_signal,axis=1)
    no_errors = (pred_output != test_label[:10])
    #print(len(pred_output))
    #print(len(test_label))
    no_errors =  no_errors.astype(int).sum()
    ber[n] = no_errors/N
    print ('SNR:',EbNodB_range[n],'BER:',ber[n])
print('Output received is:', pred_output)


Original Data:
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]

Encoded Data:
0    1.137766
1    1.137766
2   -0.334301
3    0.812698
4    1.833271
5   -1.246952
6   -0.448439
7    1.004375
8    1.833271
9   -1.942872
Name: 0, dtype: float32

Enocded & Encrypted Data:
0    <phe.paillier.EncryptedNumber object at 0x7f2f...
1    <phe.paillier.EncryptedNumber object at 0x7f2f...
2    <phe.paillier.EncryptedNumber object at 0x7f2f...
3    <phe.paillier.EncryptedNumber object at 0x7f2f...
4    <phe.paillier.EncryptedNumber object at 0x